In [28]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings("ignore")

In [94]:
data_train_full = pd.read_csv('./transformed_data/data_train_after_preprocessing.csv', sep=';')
data_test_full = pd.read_csv('./transformed_data/data_test_after_preprocessing.csv', sep=';')
data_val_full = pd.read_csv('./transformed_data/data_val_after_preprocessing.csv', sep=';')

In [96]:
data_val_full.head()

,Unnamed: 0,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,...,relief,remorse,sadness,surprise,neutral,processed_text,non_stopwords_text,punctuation_removed,cleaned_text,tokenized_text
0,0,Literally everywhere. There's nothing especial...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,literally everywhere there s nothing especiall...,literally everywhere. ' nothing especially tex...,literally everywhere nothing especially texan ...,literally everywhere nothing especially texan ...,"['literally', 'everywhere', 'nothing', 'especi..."
1,1,"[NAME] and [NAME] may be stronger, but [NAME] ...",0,0,0,0,0,0,0,0,...,0,0,0,0,1,name and name may be stronger but name just g...,"[name] [name] may stronger, [name] gone ma...",name name may stronger name gone mad lost plot...,name name may stronger name gone mad lost plot...,"['name', 'name', 'may', 'stronger', 'name', 'g..."
2,2,Ik I’m crying rn,0,0,0,0,0,0,0,0,...,0,0,1,0,0,ik i m crying rn,ik ’ crying rn,ik crying rn,ik crying rn,"['ik', 'crying', 'rn']"
3,3,"He was cut yesterday, unfortunately",0,0,0,0,0,0,0,0,...,0,0,0,0,1,he was cut yesterday unfortunately,"cut yesterday, unfortunately",cut yesterday unfortunately,cut yesterday unfortunately,"['cut', 'yesterday', 'unfortunately']"
4,4,"Nice, I saw them during the Demo and was prett...",0,1,0,0,1,0,0,0,...,0,0,0,0,0,nice i saw them during the demo and was pretty...,"nice, saw demo pretty impressed",nice saw demo pretty impressed,nice saw demo pretty impressed,"['nice', 'saw', 'demo', 'pretty', 'impressed']"


In [97]:
unuse = ['Unnamed: 0']

data_train_full = data_train_full.drop(columns = unuse)
data_test_full = data_test_full.drop(columns = unuse)
data_val_full = data_val_full.drop(columns = unuse)

In [98]:
data_train_full.head()

,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,relief,remorse,sadness,surprise,neutral,processed_text,non_stopwords_text,punctuation_removed,cleaned_text,tokenized_text
0,Don't be coy were talking about the active ing...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,don t be coy were talking about the active ing...,' coy talking active ingredients plant .,coy talking active ingredients plant,coy talking active ingredients plant,"['coy', 'talking', 'active', 'ingredients', 'p..."
1,"It's likely that they don't have much, yet sti...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,it s likely that they don t have much yet stil...,"' likely ' much, yet still make poor financ...",likely much yet still make poor financial dec...,likely much yet still make poor financial dec...,"['likely', 'much', 'yet', 'still', 'make', 'po..."
2,Sure but the news article with [NAME] vanishin...,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,sure but the news article with name vanishing ...,sure news article [name] vanishing since...,sure news article name vanishing since beginning,sure news article name vanishing since beginning,"['sure', 'news', 'article', 'name', 'vanishing..."
3,i would not vote for him,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,i would not vote for him,would vote,would vote,would vote,"['would', 'vote']"
4,Funny way to oppose fascism - since they are t...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,funny way to oppose fascism since they are the...,funny way oppose fascism - since fascists,funny way oppose fascism since fascists,funny way oppose fascism since fascists,"['funny', 'way', 'oppose', 'fascism', 'since',..."


In [99]:
data_train_full['Index'] = range(1, len(data_train_full) + 1)
data_test_full['Index'] = range(1, len(data_test_full) + 1)
data_val_full['Index'] = range(1, len(data_val_full) + 1)

In [100]:
data_train_full.set_index('Index', drop=True, append=False, inplace=False, verify_integrity=False)
data_test_full.set_index('Index', drop=True, append=False, inplace=False, verify_integrity=False)
data_val_full.set_index('Index', drop=True, append=False, inplace=False, verify_integrity=False)

,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,relief,remorse,sadness,surprise,neutral,processed_text,non_stopwords_text,punctuation_removed,cleaned_text,tokenized_text
Index,,,,,,,,,,,,,,,,,,,,,
1,Literally everywhere. There's nothing especial...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,literally everywhere there s nothing especiall...,literally everywhere. ' nothing especially tex...,literally everywhere nothing especially texan ...,literally everywhere nothing especially texan ...,"['literally', 'everywhere', 'nothing', 'especi..."
2,"[NAME] and [NAME] may be stronger, but [NAME] ...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,name and name may be stronger but name just g...,"[name] [name] may stronger, [name] gone ma...",name name may stronger name gone mad lost plot...,name name may stronger name gone mad lost plot...,"['name', 'name', 'may', 'stronger', 'name', 'g..."
3,Ik I’m crying rn,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,ik i m crying rn,ik ’ crying rn,ik crying rn,ik crying rn,"['ik', 'crying', 'rn']"
4,"He was cut yesterday, unfortunately",0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,he was cut yesterday unfortunately,"cut yesterday, unfortunately",cut yesterday unfortunately,cut yesterday unfortunately,"['cut', 'yesterday', 'unfortunately']"
5,"Nice, I saw them during the Demo and was prett...",0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,nice i saw them during the demo and was pretty...,"nice, saw demo pretty impressed",nice saw demo pretty impressed,nice saw demo pretty impressed,"['nice', 'saw', 'demo', 'pretty', 'impressed']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31680,"You should hit up [NAME], he can help.",0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,you should hit up name he can help,"hit [name], help.",hit name help,hit name help,"['hit', 'name', 'help']"
31681,"i think if you truly thought it was wrong, you...",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,i think if you truly thought it was wrong you ...,"think truly thought wrong, wouldnt defen...",think truly thought wrong wouldnt defend exte...,think truly thought wrong wouldnt defend exte...,"['think', 'truly', 'thought', 'wrong', 'wouldn..."
31682,"I am really sorry for you, but oh my [NAME] th...",0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,i am really sorry for you but oh my name this ...,"really sorry , oh [name] made cry laugh...",really sorry oh name made cry laughing hope c...,really sorry oh name made cry laughing hope c...,"['really', 'sorry', 'oh', 'name', 'made', 'cry..."


In [101]:
new_data_train_full = data_train_full.drop(['text','processed_text','non_stopwords_text','punctuation_removed','cleaned_text'],axis=1)
new_data_test_full = data_test_full.drop(['text','processed_text','non_stopwords_text','punctuation_removed','cleaned_text'],axis=1)
new_data_val_full = data_val_full.drop(['text','processed_text','non_stopwords_text','punctuation_removed','cleaned_text'],axis=1)

In [105]:
X_train = new_data_train_full["tokenized_text"]
y_train = np.asarray(new_data_train_full[new_data_train_full.columns[1:]])
X_test = new_data_test_full["tokenized_text"]
y_test = np.asarray(new_data_test_full[new_data_test_full.columns[1:]])
X_val = new_data_val_full["tokenized_text"]
y_val = np.asarray(new_data_val_full[new_data_val_full.columns[1:]])

In [106]:
X_val

0        ['literally', 'everywhere', 'nothing', 'especi...
1        ['name', 'name', 'may', 'stronger', 'name', 'g...
2                                   ['ik', 'crying', 'rn']
3                    ['cut', 'yesterday', 'unfortunately']
4           ['nice', 'saw', 'demo', 'pretty', 'impressed']
                               ...                        
31679                              ['hit', 'name', 'help']
31680    ['think', 'truly', 'thought', 'wrong', 'wouldn...
31681    ['really', 'sorry', 'oh', 'name', 'made', 'cry...
31682    ['forgot', 'add', 'hated', 'job', 'sick', 'kid...
31683            ['got', 'ta', 'admit', 'chuckled', 'one']
Name: tokenized_text, Length: 31684, dtype: object

In [107]:
print(y_test_full)

       admiration  amusement  anger  annoyance  approval  caring  confusion  \
0               0          0      0          0         0       0          0   
1               0          0      0          0         0       0          0   
2               0          0      0          0         0       0          0   
3               0          0      0          0         0       0          0   
4               0          1      0          0         1       0          0   
...           ...        ...    ...        ...       ...     ...        ...   
42240           0          0      0          0         0       0          0   
42241           1          0      0          0         0       0          0   
42242           0          0      0          0         0       0          0   
42243           0          0      0          0         0       0          0   
42244           0          0      0          0         1       0          0   

       curiosity  desire  disappointment  ...  love

###### TFIDF

In [93]:
vetorizar = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x, max_features=3000, max_df=0.85)
vetorizar.fit(X)

TfidfVectorizer(max_df=0.85, max_features=3000,
                preprocessor=<function <lambda> at 0x000002435BA315E0>,
                tokenizer=<function <lambda> at 0x000002435BA31790>)

In [108]:
# fungsi tfidf
def tfidf(x,data):
    vec = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x, max_features=3000, max_df=0.85)
    vec.fit(X)
    after_tfidf = vetorizar.transform(data)
    return after_tfidf

In [ ]:
train_tfidf = tfidf()